# 🚀 Korpusanalyse – Textkomplexität

<div class="alert alert-block alert-info"> <b> 🔔 Feinlernziel(e) dieses Kapitels</b></br>
   Sie können die Konzeption der Analyse beschreiben und andere Möglichkeiten des Korpus-Splitting entwerfen. </br>
    Sie können das Konzept eines Balkendiagramms erklären und das erstellte Diagramm interpretieren sowie die Gründe für Ihre Interpretation nennen.

</div>

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht
Im Folgenden wird die Textkomplexität der einzelnen Pressemitteilungen mit der Python-Bibliothek [readability](https://pypi.org/project/readability/) errechnet, für unterschiedliche Zeitabschnitte (Monate, Jahre) zusammengefasst und visualisiert. 
1. Einlesen des Korpus und der Metadaten
2. Errechnen der Komplexitätsmaße
5. Zusammenfassung für Zeitabschnitte und Visualisierung
6. Ergebnisse schreiben

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks</b>
<ol>
<li> Installieren der Bibliotheken </li>
<li>2. Laden der Daten (z.B. über den Command `wget` (s.u.))</li>
<li>3. Pfad zu den Daten setzen</li>
</ol>
Zum Testen: Ausführen der Zelle "load libraries" und der Sektion "Einlesen des Korpus". </br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [ ]:
# 🚀 Install libraries
! pip install textstat pandas matplotlib bokeh

In [ ]:
# import libraries for table processing and for compuation of readability metrics
from pathlib import Path
from multiprocessing import Pool, cpu_count
from functools import partial

import pandas as pd
import matplotlib.pyplot as plt
import textstat
textstat.set_lang("de")

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row, layout
from bokeh.models import ColumnDataSource, CustomJS, TextInput, Div, RadioButtonGroup, Switch, TableColumn, DataTable
# Ensure Bokeh output is displayed in the notebook
output_notebook()

## 1. Einlesen der Daten und Metadaten

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die Textdateien gespeichert werden. Der Einfachheit halber wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/dh-network/quadriga/tree/main">GitHub Repository</a>, in dem die Daten gespeichert sind, vorausgesetzt. </br>
Der Text wird aus GitHub heruntergeladen und in dem Ordner <i>../data/txt/</i> abgespeichert. </br>
Der Pfad kann in der Variable <i>text_path</i> angepasst werden. Die einzulesenden Daten müssen die Endung `.txt` haben. </br>
</details>

### TODO: here rather provide a file with the already computed results?

In [ ]:
# 🚀 Create data directory path
corpus_path = Path(r"../data/txt")
if not corpus_path.exists():
    corpus_path.mkdir()

In [ ]:
# 🚀 Create download list 
github_api_txt_dir_path = "https://github.com/dh-network/quadriga-fs-2/tree/develop/data/txt
txt_dir_info = requests.get(github_api_txt_dir_path).json()
url_list = [entry["download_url"] for entry in txt_dir_info]

# 🚀 Write download list as txt file
url_list_path = Path("github_txt_file_urls.txt")
with url_list_path.open('w') as output_txt:
    output_txt.write("\n".join(url_list))

In [ ]:
# ⚠️ Only execute, if you haven't downloaded the files yet!
# 🚀 Download all txt files – this step will take a while
! wget -i github_txt_file_urls.txt -P ../data/txt

### 1.1 Einlesen des Korpus

Setzen des Pfads:

In [ ]:
corpus_path = Path(r"../data/txt/")

Korpus einlesen:

In [ ]:
corpus = {"filename":[], "text":[]}
for fp in corpus_path.iterdir():
    corpus["filename"].append(fp.name)
    corpus["text"].append(fp.read_text())

Den ersten eingelesenen Text anzeigen lassen:

In [ ]:
print(corpus["text"][0])

### 1.2 Einlesen der Metadaten

In [ ]:
# 🚀 Create metadata directory path
metadata_dir = Path("../data/metadata")
if not metadata_dir.exists():
    metadata_dir.mkdir()

In [ ]:
# 🚀 Load the metadata file from GitHub 
! wget https://raw.githubusercontent.com/dh-network/quadriga-fs-2/tree/develop/data/metadata.csv -P ../data

Pfad zur Metadaten-Datei setzen:

In [ ]:
metadata_path = "../data/metadata.csv"

Datei einlesen:

In [ ]:
metadata_df = pd.read_csv(metadata_path, sep=",")

Ersten fünf Zeilen anzeigen lassen:

In [ ]:
metadata_df.head()

## 2. Textkomplexität berechnen 

### Metriken
Alle arbeiten mit durchschnittlicher Satzlänge, nehmen zusätzlich noch Worteigenschaften dazu:
* Flesch: Durchschnittliche Anzahl an Silben (deshalb sprachspezifisch) [0 - 100], 0 = sehr schwierig (Professionel), 100 = sehr leicht
* ARI: Wortlänge [1 -14], 1=sehr leicht, 14 = schwierig (College)
* Wiener Sachtextformel: [4-15] = Schulstufen 
* Coleman Liau Index: score = grade in school --> höher = schwieriger

Textkomplexität berechnen:

In [ ]:
# vllt dazu noch: reading time ~ text length
readability_scores = {}
with Pool(cpu_count()) as p:
    readability_scores["Flesch"] = p.map(textstat.flesch_reading_ease, corpus["text"])
    readability_scores["Wiener_Sachtextformel"] = p.map(partial(textstat.wiener_sachtextformel, variant=1), corpus["text"])
    readability_scores["ARI"] = p.map(textstat.automated_readability_index, corpus["text"])
    readability_scores["Coleman_Liau"] = p.map(textstat.coleman_liau_index, corpus["text"])

for measure, score in readability_scores.items():
    corpus[measure] = score

Ergebnisse in einem DataFrame speichern und die ersten fünf Einträge anzeigen lassen:

In [ ]:
corpus_df = pd.DataFrame(corpus)
corpus_df.head()

Korpusdaten mit dem Metadaten zusammenfügen. Die Einträge werden über den Dateinamen einander zugeordnet.

In [ ]:
metadata_df = metadata_df.merge(corpus_df)
metadata_df.head()

Die zusammengefügten Daten in einer Datei speichern:

In [ ]:
# 🚀 Create result directory path
result_path = Path(r"../results")
if not result_path.exists():
    result_path.mkdir()

In [ ]:
result_path = (r"../results")
metadata_df.to_csv("metadata_with_readability_scores.csv", index=False)

In [ ]:
# Too slow if one with entire data frame

# Convert DataFrame to ColumnDataSource
source = ColumnDataSource(metadata_df.head())
        
# Create Table Columns
columns = [TableColumn(field=col, title=col) for col in metadata_df.head().columns]

# Create DataTable
data_table = DataTable(source=source, columns=columns)

# Display DataTable
output_notebook()  # Use this to render in Jupyter Notebook
show(layout([data_table]))

## 3. Analyse

__3.3 Max und Min-Werte --> can be done with dataframe__ ?
* stimmen die Metriken bei der Bestimmung von Max und Min-Werten überein?
* Anzeigen der Sätze? 

## 3.1 Korpus filtern 
* bei Jahr (2011 - 2024)

In [ ]:
metadata_df['date'] = pd.to_datetime(metadata_df['date'])
metadata_from_2011_df = metadata_df[metadata_df.date.dt.year >= 2011]

## 3.2 Entwicklung über Zeit 
* Vergleich der Metriken-Entwicklung über die Jahre 

In [ ]:
metrics = ["Flesch", "ARI", "Coleman_Liau", "Wiener_Sachtextformel"]

In [ ]:
metric_mean_year_df = metadata_from_2011_df.groupby(pd.PeriodIndex(metadata_from_2011_df['date'], freq="Y"))[metrics].mean()

In [ ]:
metric_mean_year_df.plot()

In [ ]:
# Create subplots -- attention, they don't start with 0!
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))
axes = axes.flatten()
for i, col in enumerate(metrics):
    axes[i].plot(metric_mean_year_df.index.year, metric_mean_year_df[col], marker='o', label=col)
    axes[i].set_title(f'{col}')
    axes[i].set_xlabel('Jahre')
    axes[i].set_ylabel(col)
    axes[i].legend()
    axes[i].grid(True)

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

### 3.2 Interaktive Analysedaten vorbereiten
* filter by metric
* filter by time

In [ ]:
columns_to_keep = []
columns_to_keep.extend(metrics)
columns_to_keep.append("date")
print(columns_to_keep)

In [ ]:
metadata_filtered = metadata_from_2011_df[columns_to_keep]

In [ ]:
def plot_with_js(metadata_filtered_df: pd.DataFrame) -> None:
    """ 
    :param pd.DataFrame merged_df: The merged dataframe of all annotations
    """
    frequency_parameters = ["Y", "M", "W-MON"]
    
    metrics_to_time_frame = {}
    for metric in metrics:
        metrics_to_time_frame[metric] = {}
        for option in frequency_parameters:
            result = metadata_filtered.groupby(pd.PeriodIndex(metadata_filtered['date'], freq=option))[metric].mean()
            metrics_to_time_frame[metric][option] = {"x": result.index.to_timestamp(), "y": result.values}
    
    # Set month as default for the plot
    line_source = ColumnDataSource(data=metrics_to_time_frame["Flesch"]["Y"])

    # Create a plot
    p = figure(title=f"Lesbarkeitsmaß", x_axis_type="datetime", x_axis_label='Zeit', 
               y_axis_label='Durschnittlicher Score', width=700, height=400)
    line = p.line('x', 'y', source=line_source, line_width=2, color='blue')

    # RadioButtonGroup to select mode
    radio_button_group_time = RadioButtonGroup(labels=["Monthly", "Weekly", "Daily"], active=0)
    radio_button_group_metric = RadioButtonGroup(labels=["Flesch", "ARI", "Coleman_Liau", "Wiener_Sachtextformel"], active=0)


    # Callback to update the data based on selected mode
    callback = CustomJS(
        args=dict(
            line=line,
            sources=metrics_to_time_frame,
            radio_button_time=radio_button_group_time,
            radio_button_metric=radio_button_group_metric
        ),
        code="""                
        // Access the value of the switch
        const sources = switch_element.active ? relative_sources : absolute_sources;

        // Access the value of the RadioButtonGroup Time
        const mode = radio_button_group_time.active;
        
        // Access the value of the RadioButtonGroup
        const base = radio_button_group_metric.active;
    
        // Retrieve the selected frequency
        const freq = ["M", "W-MON", "D"][mode];

        // Retrieve the selected metric
        const metric = ["Flesch", "ARI", "Coleman_Liau", "Wiener_Sachtextformel"][base];
    
        // update data source and emit change event
        line.data_source.data = sources[metric][freq];
        line.data_source.change.emit();
    """,
    )

    # Attach the callback to both widgets
    radio_button_group_time.js_on_change('active', callback)
    radio_button_group_metric.js_on_change('active', callback)

    # Layout the RadioButtonGroup and plot
    layout = column(row(radio_button_group_metric, radio_button_group_time), p)
    show(layout)

In [ ]:
# not interactive yet, updating needs to be fixed
plot_with_js(metadata_filtered)